# <b>U.S.-Medical-Insurance-Costs


## Import libraries

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

## Make a DataFrames

In [26]:
observations = pd.read_csv('observations.csv')
species_info = pd.read_csv('species_info.csv')

## <b>Exploratory Data Analysis (EDA)

### Inspecting and cleaning data for `observations`

### Data Inspection


In [27]:
observations.head()

,scientific_name,park_name,observations
0,Vicia benghalensis,Great Smoky Mountains National Park,68
1,Neovison vison,Great Smoky Mountains National Park,77
2,Prunus subcordata,Yosemite National Park,138
3,Abutilon theophrasti,Bryce National Park,84
4,Githopsis specularioides,Great Smoky Mountains National Park,85


In [28]:
observations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23296 entries, 0 to 23295
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   scientific_name  23296 non-null  object
 1   park_name        23296 non-null  object
 2   observations     23296 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 546.1+ KB


### Missing Values and Duplicates

In [29]:
missing_values = observations.isnull().sum()
duplicates = observations.duplicated().sum()

print("missing_values: \n{ms} \n\nduplicates: {d}".format(ms = missing_values, d = duplicates))


missing_values: 
scientific_name    0
park_name          0
observations       0
dtype: int64 

duplicates: 15


### Drop Duplicates

In [30]:
observations.drop_duplicates(inplace=True)
duplicates = observations.duplicated().sum()

print("duplicates: {0}\n".format(duplicates))


# Reindex the DataFrame
observations.reset_index(drop=True, inplace=True)

observations.info()

duplicates: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23281 entries, 0 to 23280
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   scientific_name  23281 non-null  object
 1   park_name        23281 non-null  object
 2   observations     23281 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 545.8+ KB


## Inspecting and cleaning data for `species_info`

### Data Inspection

In [31]:
species_info.head()

,category,scientific_name,common_names,conservation_status
0,Mammal,Clethrionomys gapperi gapperi,Gapper's Red-Backed Vole,NaN
1,Mammal,Bos bison,"American Bison, Bison",NaN
2,Mammal,Bos taurus,"Aurochs, Aurochs, Domestic Cattle (Feral), Dom...",NaN
3,Mammal,Ovis aries,"Domestic Sheep, Mouflon, Red Sheep, Sheep (Feral)",NaN
4,Mammal,Cervus elaphus,Wapiti Or Elk,NaN


In [32]:
species_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5824 entries, 0 to 5823
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   category             5824 non-null   object
 1   scientific_name      5824 non-null   object
 2   common_names         5824 non-null   object
 3   conservation_status  191 non-null    object
dtypes: object(4)
memory usage: 182.1+ KB


### Missing Values and Duplicates

In [33]:
missing_values = species_info.isnull().sum()
duplicates = species_info.duplicated().sum()

print("missing_values: \n{ms} \n\nduplicates: {d}".format(ms = missing_values, d = duplicates))

missing_values: 
category                  0
scientific_name           0
common_names              0
conservation_status    5633
dtype: int64 

duplicates: 0


### Dealing with Missing Values

The '`conservation_status`' column contains information about the current conservation status of species. For species that are not under the threat of extinction and are in a normal state, we will use the status '`Not Threatened`'.

In [42]:
species_info['conservation_status'].fillna('Not Threatened', inplace=True)

missing_values = species_info.isnull().sum()
print("missing values:\n{0}".format(missing_values))

missing values:
category               0
scientific_name        0
common_names           0
conservation_status    0
dtype: int64


## <b> Numerical summarization

### For `observations`

In [ ]:
observations.describe()

,observations
count,23281.000000
mean,142.280357
std,69.885439
min,9.000000
25%,86.000000
50%,124.000000
75%,195.000000
max,321.000000


### For `species_info`

In [43]:
species_info.describe()

,category,scientific_name,common_names,conservation_status
count,5824,5824,5824,5824
unique,7,5541,5504,5
top,Vascular Plant,Castor canadensis,Brachythecium Moss,Not Threatened
freq,4470,3,7,5633


## <b> Data Visualization

### For `observations`